In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'c:\\Users\\teck1\\OneDrive\\Documents\\GitHub\\ai'

In [39]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class PrepareModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int
    params_image_height: int
    params_image_width: int
    params_sequence: int
    params_classes_list: list

In [34]:
from DetectAnalytics.constants import *
from DetectAnalytics.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareModelConfig:
        config = self.config.preparing_model
        
        create_directories([config.root_dir])

        prepare_model_config = PrepareModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES,
            params_image_height=self.params.IMAGE_HEIGHT,
            params_image_width=self.params.IMAGE_WIDTH,
            params_sequence=self.params.SEQUENCE,
            params_classes_list=self.params.CLASSES_LIST
        )

        return prepare_model_config

In [ ]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

import shutil
import math
import random
import datetime as dt
from collections import deque
import matplotlib.pyplot as plt
plt.style.use("seaborn")

%matplotlib inline

In [40]:
class PrepareBaseModel:
    def __init__(self, config: PrepareModelConfig):
        self.config = config


    
    def get_base_model(self):

        self.model = tf.keras.applications.mobilenet_v2.MobileNetV2(
            include_top=self.config.params_include_top,
            weights=self.config.params_weights
        )

        self.model.trainable = True
        for layer in self.model.layers[:-40]:
            layer.trainable=False
            
        self.save_model(path=self.config.base_model_path, model=self.model)


    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate, image_size):

        sequential = tf.keras.models.Sequential()
        sequential.add(tf.keras.layers.Input(shape = (image_size)))
        sequential.add(tf.keras.layers.TimeDistributed(model))
        sequential.add(tf.keras.layers.Dropout(0.25))
        sequential.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()))
        lstm_fw = tf.keras.layers.LSTM(units=32)
        lstm_bw = tf.keras.layers.LSTM(units=32, go_backwards=True)
        sequential.add(tf.keras.layers.Bidirectional(lstm_fw, backward_layer = lstm_bw))
        sequential.add(tf.keras.layers.Dropout(0.25))
        sequential.add(tf.keras.layers.Dense(256, activation='relu'))
        sequential.add(tf.keras.layers.Dropout(0.25))
        sequential.add(tf.keras.layers.Dense(128, activation='relu'))
        sequential.add(tf.keras.layers.Dropout(0.25))
        sequential.add(tf.keras.layers.Dense(64, activation='relu'))
        sequential.add(tf.keras.layers.Dropout(0.25))
        sequential.add(tf.keras.layers.Dense(32, activation='relu'))
        sequential.add(tf.keras.layers.Dropout(0.25))

        sequential.add(tf.keras.layers.Dense(classes, activation='softmax'))

        sequential.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        sequential.summary()
        return sequential
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate,
            image_size=self.config.params_image_size
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    

In [41]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2023-12-17 20:10:13,393: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-17 20:10:13,396: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-17 20:10:13,401: INFO: common: created directory at: artifacts]
[2023-12-17 20:10:13,401: INFO: common: created directory at: artifacts/preparing_model]
[2023-12-17 20:10:13,402: WARNING: mobilenet_v2: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.]
[2023-12-17 20:10:14,308: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]


c:\Users\teck1\miniconda3\envs\detectai\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_10 (TimeD  (None, 16, 2, 2, 1280)    2257984   
 istributed)                                                     
                                                                 
 dropout_30 (Dropout)        (None, 16, 2, 2, 1280)    0         
                                                                 
 time_distributed_11 (TimeD  (None, 16, 5120)          0         
 istributed)                                                     
                                                                 
 bidirectional_5 (Bidirecti  (None, 64)                1319168   
 onal)                                                           
                                                                 
 dropout_31 (Dropout)        (None, 64)                0         
                                                     